In [ ]:
import main_mfrl_REINFORCE_vanilla as Vanilla
import main_mfrl_REINFORCE_recurrent as Recurrent
import main_mfrl_REINFORCE_RA2C as RA2C
from mfrl_lib.lib import *

import torch
from torch.distributions import Categorical
import pandas as pd
from tqdm import tqdm

# if GPU is to be used
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

topology.show_adjacency_matrix()

In [16]:
agents = [RA2C.Agent(topology, i) for i in range(node_n)]

# Load the trained models
for i in range(topology.n):
    agents[i].pinet.load_state_dict(torch.load(f"models/RA2C_agent_{i}.pth", map_location=device))

df = pd.DataFrame()
rewards = []
for n_epi in tqdm(range(20)):
    # Reset the environment
    episode_utility = 0
    states = [torch.from_numpy(agent.env.reset()[0].astype('float32')).unsqueeze(0).to(device) for agent in agents]
    probs = [None]*node_n
    h = [torch.zeros(1, 1, 32).to(device) for _ in range(node_n)]
    c = [torch.zeros(1, 1, 32).to(device) for _ in range(node_n)]
    for t in range(MAX_STEPS):
        # Calculate the action for each agent
        actions = [None]*node_n
        for i in range(node_n):
            with torch.no_grad():
                probs[i], _, h[i], c[i] = agents[i].pinet(torch.tensor(states[i]), 
                                                          h[i], 
                                                          c[i])
                action = Categorical(probs[i]).sample().item()
                actions.append(action)
        print(actions)

  0%|          | 0/20 [00:00<?, ?it/s]/var/folders/7x/6zct7wks35n21xhkxwrwggvr0000gn/T/ipykernel_59646/4284258986.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probs[i], _, h[i], c[i] = agents[i].pinet(torch.tensor(states[i]),
  0%|          | 0/20 [00:00<?, ?it/s]


ValueError: Expected parameter probs (Tensor of shape (1, 1, 2)) of distribution Categorical(probs: torch.Size([1, 1, 2])) to satisfy the constraint Simplex(), but found invalid values:
tensor([[[-0.4640,  1.4640]]], device='mps:0')